<a href="https://colab.research.google.com/github/verma-saloni/Thesis-Work/blob/main/4_5_22_1_Sbert_embeddings_extra_xgboost_politifact.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install -U xgboost sentence-transformers wandb

In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import re

from sentence_transformers import SentenceTransformer 

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import xgboost as xgb

import wandb
from wandb.xgboost import WandbCallback

In [ ]:
metrics = [accuracy_score, f1_score, precision_score, recall_score]

def get_name(score_func):
    return score_func.__name__.split("_")[0]


Load data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
base_dir = Path("/content/drive/MyDrive/")
assert base_dir.exists()

In [ ]:
df_fake = pd.read_csv(base_dir/'politifact_fake.csv')
df_real = pd.read_csv(base_dir/'politifact_real.csv')
df_fake['label']=0
df_real['label']=1
df_fake.count()

id           432
news_url     428
title        432
tweet_ids    392
label        432
dtype: int64

In [ ]:
df = df_fake.append(df_real)
df.reset_index(drop=True, inplace=True)

In [ ]:
df.head()

,id,news_url,title,tweet_ids,label
0,politifact15014,speedtalk.com/forum/viewtopic.php?t=51650,BREAKING: First NFL Team Declares Bankruptcy O...,937349434668498944\t937379378006282240\t937380...,0
1,politifact15156,politics2020.info/index.php/2018/03/13/court-o...,Court Orders Obama To Pay $400 Million In Rest...,972666281441878016\t972678396575559680\t972827...,0
2,politifact14745,www.nscdscamps.org/blog/category/parenting/467...,UPDATE: Second Roy Moore Accuser Works For Mic...,929405740732870656\t929439450400264192\t929439...,0
3,politifact14355,https://howafrica.com/oscar-pistorius-attempts...,Oscar Pistorius Attempts To Commit Suicide,886941526458347521\t887011300278194176\t887023...,0
4,politifact15371,http://washingtonsources.org/trump-votes-for-d...,Trump Votes For Death Penalty For Being Gay,915205698212040704\t915242076681506816\t915249...,0


Compute emebeddings

In [ ]:
embedding_file = Path("/content/drive/MyDrive/sbert_embeddings.npy")

if embedding_file.exists():
    embeddings = np.load(embedding_file)
else:
    model_id = "all-mpnet-base-v2"
    model = SentenceTransformer(model_id)

    sentences = df.title.to_list()
    embeddings = model.encode(sentences, show_progress_bar=True)
    np.save(embedding_file, embeddings)

Extra features: add more features here, if all looks good. 

Source

In [ ]:
source = df.news_url.str.replace("^(https://)?web.archive.org/web/\d+/", "", regex=True)

In [ ]:
source = source.str.replace("^(https?://)?(www.)?", "", regex=True)

In [ ]:
source = source.str.replace("\.(com|info|org|gov|tv|us|news|me|co.uk|net|club|co|live|edu|xyz|site|life|ru|online|tk|website|pw|one|world|mil).*$", "", regex=True)

In [ ]:
len(source), len(source.unique())


(1056, 526)

Number of retweets

In [ ]:
df["num_retweets"] = df.tweet_ids.fillna("").map(lambda x: len(str(x).split("\t")) if len(x) else 0)
df["log_num_retweets"] = np.log(df.num_retweets.to_numpy() + 1)

In [ ]:
df.describe()

,label,num_retweets,log_num_retweets
count,1056.000000,1056.000000,1056.000000
mean,0.590909,552.571970,3.363832
std,0.491899,2126.317823,2.704599
min,0.000000,0.000000,0.000000
25%,0.000000,1.000000,0.693147
50%,1.000000,30.000000,3.433987
75%,1.000000,246.250000,5.510398
max,1.000000,29060.000000,10.277152


More to come here..

Training

In [ ]:
X = np.concatenate([embeddings, df.log_num_retweets.to_numpy()[..., None]], axis=1)
y = df.label.to_numpy()

X.shape, y.shape

((1056, 769), (1056,))

In [ ]:
skf = StratifiedKFold(shuffle=True, random_state=124)

In [ ]:
def train(train_idx, test_idx, params):

    # training
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dtest = xgb.DMatrix(X_test, label=y_test)
    watchlist = [(dtrain,'train'), (dtest,'eval')]
    clf = xgb.train(params, dtrain, num_boost_round=100, early_stopping_rounds=None, evals=watchlist, callbacks=[WandbCallback()])
    #evaluation
    probs = clf.predict(dtest)
    y_pred = (probs > 0.5).astype(int)
    eval_results = {get_name(f):f(y_pred=y_pred, y_true=y_test) for f in metrics}
    wandb.log(eval_results)
    wandb.log({"conf_mat" : wandb.plot.confusion_matrix(probs=None,
                            y_true=y_test, preds=y_pred,
                            class_names=["Fake", "Real"])})

In [ ]:
params = {
    "objective":'binary:logistic',
    "seed":124
}

In [ ]:
for fold_id, (train_idx, test_idx) in enumerate(skf.split(X, y)):
    with wandb.init(entity="saloniteam", project="fnd", group="sbert-mpnet-v2+lnt-xgb", name=f"sbert-mpnet-v2+xgb-fold-{fold_id}") as run:
        train(train_idx, test_idx, params)

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


[0]	train-logloss:0.49849	eval-logloss:0.58610
[1]	train-logloss:0.37901	eval-logloss:0.50565
[2]	train-logloss:0.29266	eval-logloss:0.44708
[3]	train-logloss:0.23140	eval-logloss:0.41384
[4]	train-logloss:0.18511	eval-logloss:0.38660
[5]	train-logloss:0.15363	eval-logloss:0.36962
[6]	train-logloss:0.12667	eval-logloss:0.34499
[7]	train-logloss:0.10608	eval-logloss:0.33883
[8]	train-logloss:0.09053	eval-logloss:0.32773
[9]	train-logloss:0.07714	eval-logloss:0.32147
[10]	train-logloss:0.06679	eval-logloss:0.31234
[11]	train-logloss:0.05845	eval-logloss:0.30866
[12]	train-logloss:0.05152	eval-logloss:0.30468
[13]	train-logloss:0.04586	eval-logloss:0.29732
[14]	train-logloss:0.04089	eval-logloss:0.29311
[15]	train-logloss:0.03713	eval-logloss:0.28604
[16]	train-logloss:0.03365	eval-logloss:0.28250
[17]	train-logloss:0.03086	eval-logloss:0.27879
[18]	train-logloss:0.02857	eval-logloss:0.27749
[19]	train-logloss:0.02652	eval-logloss:0.27341
[20]	train-logloss:0.02481	eval-logloss:0.27075
[2

accuracy,▁
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
eval-logloss,█▅▃▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
f1,▁
precision,▁
recall,▁
train-logloss,█▅▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
accuracy,0.89623
epoch,99
f1,0.91406
precision,0.89313


wandb: Currently logged in as: saloniteam (use `wandb login --relogin` to force relogin)


...

accuracy,▁
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
eval-logloss,█▆▄▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
f1,▁
precision,▁
recall,▁
train-logloss,█▅▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
accuracy,0.90047
epoch,99
f1,0.91566
precision,0.91935


[0]	train-logloss:0.49186	eval-logloss:0.57930
[1]	train-logloss:0.36852	eval-logloss:0.51609
[2]	train-logloss:0.28510	eval-logloss:0.46525
[3]	train-logloss:0.22635	eval-logloss:0.43531
[4]	train-logloss:0.18224	eval-logloss:0.40688
[5]	train-logloss:0.14910	eval-logloss:0.39505
[6]	train-logloss:0.12409	eval-logloss:0.39153
[7]	train-logloss:0.10269	eval-logloss:0.37595
[8]	train-logloss:0.08609	eval-logloss:0.37879
[9]	train-logloss:0.07327	eval-logloss:0.36113
[10]	train-logloss:0.06339	eval-logloss:0.35517
[11]	train-logloss:0.05553	eval-logloss:0.35670
[12]	train-logloss:0.04906	eval-logloss:0.35280
[13]	train-logloss:0.04363	eval-logloss:0.34793
[14]	train-logloss:0.03904	eval-logloss:0.34494
[15]	train-logloss:0.03527	eval-logloss:0.34186
[16]	train-logloss:0.03224	eval-logloss:0.33630
[17]	train-logloss:0.02958	eval-logloss:0.33333
[18]	train-logloss:0.02727	eval-logloss:0.33390
[19]	train-logloss:0.02539	eval-logloss:0.33458
[20]	train-logloss:0.02361	eval-logloss:0.33476
[2

accuracy,▁
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
eval-logloss,█▅▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁
f1,▁
precision,▁
recall,▁
train-logloss,█▅▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
accuracy,0.87678
epoch,99
f1,0.89344
precision,0.91597


[0]	train-logloss:0.50642	eval-logloss:0.58410
[1]	train-logloss:0.38076	eval-logloss:0.52984
[2]	train-logloss:0.29592	eval-logloss:0.48506
[3]	train-logloss:0.23673	eval-logloss:0.45727
[4]	train-logloss:0.18913	eval-logloss:0.42034
[5]	train-logloss:0.15454	eval-logloss:0.39430
[6]	train-logloss:0.12735	eval-logloss:0.38000
[7]	train-logloss:0.10566	eval-logloss:0.37681
[8]	train-logloss:0.08931	eval-logloss:0.38139
[9]	train-logloss:0.07614	eval-logloss:0.38026
[10]	train-logloss:0.06559	eval-logloss:0.37055
[11]	train-logloss:0.05741	eval-logloss:0.36313
[12]	train-logloss:0.05115	eval-logloss:0.35830
[13]	train-logloss:0.04567	eval-logloss:0.35464
[14]	train-logloss:0.04086	eval-logloss:0.34977
[15]	train-logloss:0.03711	eval-logloss:0.34289
[16]	train-logloss:0.03367	eval-logloss:0.34921
[17]	train-logloss:0.03085	eval-logloss:0.35037
[18]	train-logloss:0.02842	eval-logloss:0.35486
[19]	train-logloss:0.02637	eval-logloss:0.35609
[20]	train-logloss:0.02472	eval-logloss:0.35815
[2

accuracy,▁
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
eval-logloss,█▅▂▂▂▁▁▁▁▁▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▃▂▂▂▂▂▂▃▂▂▃▃
f1,▁
precision,▁
recall,▁
train-logloss,█▅▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
accuracy,0.88152
epoch,99
f1,0.90119
precision,0.89062
